# 10.1 практика, приклад з конспекту...

згорткові нейронні мережі...

###### __Створення моделі__

- https://keras.io/api/layers/convolution_layers/convolution2d/

In [2]:
from keras import layers
from keras import models



model = models.Sequential([
                            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # 1 - gray, 3 - RGB
                            layers.MaxPooling2D((2, 2)),

                            layers.Conv2D(64, (3, 3), activation='relu'),
                            layers.MaxPooling2D((2, 2)),

                            layers.Conv2D(64, (3, 3), activation='relu'),

                            layers.Flatten(),
                            layers.Dense(64, activation='relu'),

                            layers.Dense(10, activation='softmax')
                            ])

всі шари, Conv2D та MaxPooling2D, генерують тривимірний тензор із формою (висота, ширина, канали). <br>
Вимірювання ширини та висоти `стискаються` зі зростанням глибини мережі. `Кількість каналів` управляється першим аргументом, переданим у шари Conv2D (`32` або `64`).

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

`Flatten` перетворює вхідний тензор розміру 3×3×64, згенерованого на попередньому шарі в одновимірний вектор розміру 3×3×64=576

`Conv2D`. Основна функція даного шару це виконання операції згортки. __Згортка__ (`convolution`) - це операція, що показує наскільки одна функція схожа на відбиту та зрушену копію іншої. У разі обробки зображень згортка - це операція `обчислення нового значення заданого пікселя, при якій враховуються значення сусідів`, що його оточують. Головним елементом згортки є матриця, яка називається `ядром згортки`. У загальному випадку ядро може мати довільні розміри, але на практиці `найчастіше` використовують матриці розміру `3×3`. 

Повнозв'язковий шар `Dense` вивчає глобальні шаблони у просторі ознак `залучаючи всі пікселі зображення`. У той час як згортковий шар `Conv2D` "бачить" `локальні`.

Паттерни, які вивчає згорткова нейромережа, є `інваріантними по відношенню до операції переносу`. Одного разу вивчивши шаблон мережа здатна розпізнати його `в будь-якому місці` на зображенні. У той час як повнозв'язковій мережі довелося б вивчити шаблон знову опиниться він в іншому місці. Тому таким мережам `потрібно менше навчальних зразків` для отримання уявлень, що мають силу узагальнення.

Вони `можуть вивчати просторові ієрархії шаблонів`. `Перший` згортковий `шар реагує на невеликі локальні шаблони`, такі як краї, `другий - на більші, що складаються з ознак першого шару` як, наприклад, форми об'єктів. Це дозволяє згортковим нейронним мережам `ефективно вивчати все більш складні та абстрактні` візуальні уявлення.

Вхідні тривимірні тензоры, з яких виконується операція згортки, називаються `картами ознак`. Зазвичай вони мають три виміри: висота, ширина, а також `глибина`, яку часто інтерпретують як `канали`.

Операція згортання витягує шаблони зі своєї вхідної карти ознак та застосовує `однакові перетворення до всіх шаблонів` при цьому `генеруючи вихідну картку ознак`.

її глибина може мати `будь-яку розмірність`, тому що вихідна глибина є `параметром шару`, та різні канали на цій осі глибини більше не відповідають конкретним кольорам, як у вхідних даних у форматі RGB, скоріше вони відповідають `"фільтрам"`.

(32 фільтри. Кожен з них містить матрицю розміру 26×26).

Є два `ключові параметри згортки`:<br>
- `Розмір вікна` згортки. Зазвичай використовуються вікна 3×3 або 5×5.
- `Глибина вихідної карти ознак` - кількість фільтрів, що генеруються згорткою. 

Перетворення відбувається завдяки `множенню вхідного тензора на матрицю ваг, одержувану під час навчання`. Таким чином, отримані вектори в кінцевому результаті збираються в тривимірну вихідну картку з розміром `height`, `width`, `output_depth`.

Вихідні ширина та висота можуть відрізнятися від вхідних. Причиною цього можуть бути:<br>
- ефекти меж та доповнення (`padding`);
- обраний крок згортки (`stride`).

Обидва ці ефекти призводять до зменшення розширення(роздільної здатності) вихідного тензора, що може чинити як позитивний, так і негативний вплив. 

При використанні шарів `Conv2D` доповнення налаштовується за допомогою `аргументу padding`, який приймає два значення: `valid`, що означає `відсутність доповнення` (використовуються лише допустимі розташування вікна), та `same`, що означає "доповнити до однаковості розмірів вихідної карти ознак (ширину та висоту) з вхідною". `За замовчуванням` аргумент padding має значення `valid`.

`крок згортки` - значення, не яке зміщується її вікно (за замовчуванням `1`). Є можливість визначати згортки з пробілами (`strided convolutions`) - згортки з кроком `більше 1`.
Використання кроку 2 означає `зменшення ширини та висоти карти ознак за рахунок зменшення дозволу` в два рази (в доповнення до будь-яких змін, спричинених ефектами меж). Згортки з пробілами рідко використовуються на практиці.

Для зменшення роздільної здатності карти ознак замість кроку часто також використовується операція __макс пулінгу__ (`max pooling`). Ця операція полягає у `виборі максимального значення пікселя серед усіх його сусідів`. Концептуально це дуже нагадує згортку, але замість лінійного перетворення частини зображення з ядром згортки ми виконуємо `агресивне зменшення дозволу карти` за рахунок вибору `максимального значення` пікселя серед усіх, що потрапили `у деяке вікно`. Головна відмінність від згортки полягає в тому, що вибір максимального значення із сусідніх зазвичай проводиться з вікном `2×2` і кроком `2`, щоб зменшити роздільну здатність карти ознак `в два рази`. У той час як згортка, як правило, виконується з вікном 3×3 з кроком 1.

з якою метою взагалі проводиться зниження дозволу карти ознак?<br>
- ...Високорівневі шаблони, вивчені за допомогою згорткової нейронної мережі, будуть занадто малі в порівнянні з початковими даними, чого може виявитися недостатньо для навчання класифікації цифр. ...
- Заключна карта ознак може мати 22×22×64= `30976` коефіцієнтів на зразок. Якби зробили її плоскою щоб накласти зверху шар `Dense розміром 512`, він мав би `15,8 мільйона параметрів`. Це надто багато для такої маленької моделі і в результаті призведе до `інтенсивного перенавчання`.

Іншими словами, зменшення дозволу використовується для зменшення кількості коефіцієнтів у карті ознак та впровадження ієрархій просторових фільтрів за рахунок створення послідовних шарів згортки для перегляду все більш великих вікон.

Крім того, замість вибору максимального значення можна використовувати операцію вибору середнього значення по сусіднім елементам (`average pooling`), коли кожний локальний шаблон перетворюється шляхом `взяття середнього значення для кожного каналу` у шаблоні замість максимального. Проте `операція вибору максимального значення зазвичай дає найкращі результати`. Причина в тому, що ознаки, як правило, кодують просторову присутність деякого шаблону або поняття в різних клітинах карти ознак, тому максимальна присутність ознак набагато інформативніша, ніж середня.

###### __Навчання__

In [4]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 9s 9ms/step - loss: 0.1765 - accuracy: 0.9443
Epoch 2/5
938/938 [==============================] - 9s 9ms/step - loss: 0.0466 - accuracy: 0.9856
Epoch 3/5
938/938 [==============================] - 9s 10ms/step - loss: 0.0333 - accuracy: 0.9898
Epoch 4/5
938/938 [==============================] - 9s 9ms/step - loss: 0.0252 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 10s 10ms/step - loss: 0.0206 - accuracy: 0.9937


обчислимо точність класифікації мережі

In [5]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0387 - accuracy: 0.9883
0.9883000254631042


###### __боротьба з перенавчанням__